# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.


client = OpenAI()
llm_model = "gpt-3.5-turbo"


In [5]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content


In [58]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [59]:
df

,product name,review of the product
0,Laptop X200,Excellent performance and battery life.
1,Smartphone Y10,Great value for the price with decent camera q...
2,Headphones Z300,Comfortable fit and superb sound quality.
3,Smartwatch A2,Sleek design with a wide range of features.
4,Tablet B1,"Lightweight and fast, but the screen could be ..."
5,Camera C400,Alta resolución y buen rendimiento en condicio...
6,Printer D100,Reliable with fast printing speeds.
7,Monitor E50,Clear display with accurate colors.
8,Keyboard F200,Ergonomic design and responsive keys.
9,Mouse G5,"Compact and smooth, ideal for everyday use."


In [57]:
df.head(10)

,product name,review of the product
0,Laptop X200,Excellent performance and battery life.
1,Smartphone Y10,Great value for the price with decent camera q...
2,Headphones Z300,Comfortable fit and superb sound quality.
3,Smartwatch A2,Sleek design with a wide range of features.
4,Tablet B1,"Lightweight and fast, but the screen could be ..."
5,Camera C400,High resolution and good low-light performance.
6,Printer D100,Reliable with fast printing speeds.
7,Monitor E50,Clear display with accurate colors.
8,Keyboard F200,Ergonomic design and responsive keys.
9,Mouse G5,"Compact and smooth, ideal for everyday use."


## LLMChain

In [13]:
from langchain.chat_models import ChatOpenAI

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [14]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [15]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [27]:
# chain = LLMChain(llm=llm, prompt=prompt)
chain = prompt | llm

In [30]:
product = "Queen Size Sheet Set"
chain.invoke(product).content

'"Regal Linens"'

## SimpleSequentialChain

In [31]:
from langchain.chains import SimpleSequentialChain, LLMChain

In [36]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(prompt=first_prompt,llm=llm)

In [37]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(prompt=second_prompt, llm=llm)

In [38]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [39]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
"Royalty Linens"
Royalty Linens offers luxurious and high-quality linens for your home, enhancing comfort and style with their elegant designs.

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': 'Royalty Linens offers luxurious and high-quality linens for your home, enhancing comfort and style with their elegant designs.'}

## SequentialChain

In [40]:
from langchain.chains import SequentialChain

In [41]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain( prompt=first_prompt, llm=llm,
                     output_key="English_Review"
                    )


In [42]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain( prompt=second_prompt, llm=llm, 
                     output_key="summary"
                    )


In [43]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [44]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [45]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [61]:
review = df['review of the product'][5]
# review
result = overall_chain.invoke(review)
# result = chain_one.invoke(review)

# print(result.keys())
from pprint import pprint
pprint(result)



> Entering new SequentialChain chain...

> Finished chain.
{'English_Review': 'High resolution and good performance in low light '
                   'conditions.',
 'Review': 'Alta resolución y buen rendimiento en condiciones de poca luz.',
 'followup_message': 'Gracias por compartir esa información. Sería bueno saber '
                     'si el producto también tiene una buena durabilidad en '
                     'general.',
 'summary': 'The product has high resolution and performs well in low light '
            'conditions.'}
